# Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [4]:
from secrets import IEX_CLOUD_API_TOKEN

#### Sample of aapl stock get request

In [9]:
ticker = 'AAPL'
endpoint = 'https://sandbox.iexapis.com'
api_url = endpoint + f'/stable/stock/{ticker}/quote/?token={IEX_CLOUD_API_TOKEN}'

In [11]:
# get data
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'DASQNA', 'calculationPrice': 'close', 'open': 0, 'openTime': None, 'openSource': 'floicfai', 'close': 0, 'closeTime': None, 'closeSource': 'aficlfoi', 'high': 0, 'highTime': None, 'highSource': None, 'low': 0, 'lowTime': None, 'lowSource': None, 'latestPrice': 148.54, 'latestSource': 'Close', 'latestTime': 'August 9, 2021', 'latestUpdate': 1688016332120, 'latestVolume': None, 'iexRealtimePrice': 149.13, 'iexRealtimeSize': 10, 'iexLastUpdated': 1656291867245, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 149.85, 'previousVolume': 55074086, 'change': -0.05, 'changePercent': -0.00035, 'volume': None, 'iexMarketPercent': 61.80080975839931, 'iexVolume': 600701, 'avgTotalVolume': 85858404, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexAskPrice': 0, 'ie

# Data Prep

In [27]:
columns_ = ['ticker', 'stock_price', 'market_cap', 'no_of_shares_tobuy']

df = pd.DataFrame(columns=columns_)

In [28]:
for ticker in stocks['Ticker']:
    api_url = endpoint + f'/stable/stock/{ticker}/quote/?token={IEX_CLOUD_API_TOKEN}pos_size   data = requests.get(api_url).json()
    
    df =\
    df.append(
        pd.Series([
            data['symbol'],
            data['latestPrice'],
            data['marketCap'],
            np.nan
        ],
        index=columns_
        ),
        ignore_index=True
    )

In [29]:
df

,ticker,stock_price,market_cap,no_of_shares_tobuy
0,A,164.36,49461198037,NaN
1,AAL,20.96,13920266995,NaN
2,AAP,205.16,13982209970,NaN
3,AAPL,150.22,2460331768107,NaN
4,ABBV,119.42,204304238952,NaN
...,...,...,...,...
500,YUM,137.10,39763372787,NaN
501,ZBH,154.14,30856554755,NaN
502,ZBRA,587.78,31528420560,NaN
503,ZION,55.91,8876949848,NaN


In [24]:
# batch execution can be done as well to reduce time required for data fetch
def batch(li, n):
    # get n sized batches from list
    for i in li:
        yield lst[i:i+n]
# fetch batch based get api request from iex cloud
# endpoint + batch_req_url

In [33]:
# equal weight pf
pf = float(input('Enter Portfolio size:'))

pos_size = pf/df.shape[0]

df['no_of_shares_tobuy'] = df['stock_price'].apply(lambda x: math.floor(pos_size/x))

Enter Portfolio size: 1000000


In [43]:
writer = pd.ExcelWriter('./recommended_trades.xlsx', engine='xlsxwriter')
df.to_excel(writer, 'Recommended Trades', index=False)

## Quantitative Momentum Strategy

## Quantitative Value Strategy